In [1]:
import requests
import json
import pandas as pd
from tqdm.notebook import tqdm

#Функции для скрапинга
def get_catalogs_wb():
    #получение каталога вб
    url = 'https://www.wildberries.ru/webapi/menu/main-menu-ru-ru.json'
    headers = {'Accept': "*/*", 'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    response = requests.get(url, headers=headers)
    data = response.json()
    data_list = []
    for d in data:
        try:
            for child in d['childs']:
                try:   
                    category_name = child['name']
                    category_id = child['id']
                    category_url = child['url']
                    shard = child['shard']
                    query = child['query']
                    data_list.append({
                        'categoty_id' : category_id,
                        'category_name': category_name,
                        'category_url': category_url,
                        'shard': shard,
                        'query': query})
                except:
                    continue
                try:
                    for sub_child in child['childs']:
                        category_name = sub_child['name']
                        category_id = sub_child['id']
                        category_url = sub_child['url']
                        shard = sub_child['shard']
                        query = sub_child['query']
                        data_list.append({
                            'categoty_id' : category_id,
                            'category_name': category_name,
                            'category_url': category_url,
                            'shard': shard,
                            'query': query})
                except:
                    # print(f'не имеет дочерних каталогов *{i["name"]}*')
                    continue
        except:
            # print(f'не имеет дочерних каталогов *{d["name"]}*')
            continue
    return data_list

def search_category_in_catalog(url, catalog_list):
    #пишем проверку пользовательской ссылки на наличии в каталоге
    try:
        for catalog in catalog_list:
            if catalog['category_url'] == url.split('https://www.wildberries.ru')[-1]:
                print(f'найдено совпадение: {catalog["category_name"]}')
                id_category = catalog['categoty_id']
                name_category = catalog['category_name']
                shard = catalog['shard']
                query = catalog['query']
                return id_category, name_category, shard, query
            else:
                # print('нет совпадения')
                pass
    except:
        print('Данный раздел не найден!')

def get_content(shard, query, low_price=None, top_price=None):
    # вставляем ценовые рамки для уменьшения выдачи, wb отдает только 100 страниц
    headers = {'Accept': "*/*", 'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    data_list = []
    for page in tqdm(range(1, 101)):
        url = f'https://catalog.wb.ru/catalog/{shard}/catalog?appType=1&curr=rub&dest=-1075831,-77677,-398551,12358499' \
              f'&locale=ru&page={page}&priceU={low_price * 100};{top_price * 100}' \
              f'&reg=0&regions=64,83,4,38,80,33,70,82,86,30,69,1,48,22,66,31,40&sort=popular&spp=0&{query}'
        r = requests.get(url, headers=headers)
        data = r.json()
        if len(get_data_from_json(data)) > 0:
            data_list.extend(get_data_from_json(data))
        else:
            break
    return data_list

def supplier_inf(data):
    sup_list = []
    headers = {'Accept': "*/*", 'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    for i in tqdm(range(0,len(data))):
        key = data[i]
        url = f'https://www.wildberries.ru/webapi/seller/data/short/{key}'
        r = requests.get(url, headers=headers)
        all_l = r.json()
        sup_list.append(all_l)
    return sup_list

def get_data_from_json(json_file):
    #извлекаем из json интересующие нас данные
    data_list = []
    for data in json_file['data']['products']:
        try:
            price = int(data["priceU"] / 100)
        except:
            price = 0
        data_list.append({
            'Наименование категории запроса': name_category,
            'id категории запроса': id_category,
            'Наименование товара': data['name'],
            'id товара': data['id'],
            'Скидка': data['sale'],
            'Цена': price,
            'Цена со скидкой': int(data["salePriceU"] / 100),
            'Бренд': data['brand'],
            'id бренда': int(data['brandId']),
            'id магазина': int(data['supplierId']),
            'id основной категории': int(data['subjectId']),
            'id категории': int(data['subjectParentId']),
            'Отзывы': data['feedbacks'],
            'Рейтинг': data['rating'],
            'Ссылка': f'https://www.wildberries.ru/catalog/{data["id"]}/detail.aspx?targetUrl=BP'
        })
    return data_list

def save_excel(data, filename):
    #сохранение результата в excel файл
    df = pd.DataFrame(data)
    writer = pd.ExcelWriter(f'{filename}.xlsx')
    df.to_excel(writer, 'data')
    writer.save()
    print(f'Все сохранено в {filename}.xlsx')

def save_json(data, filename):
    #сохранение результата в json файл
    with open(f'{filename}.json', 'w', encoding='UTF-8') as file:
      json.dump(data, file, indent=2, ensure_ascii=False)
      print(f'Данные сохранены в {filename}.json')

In [2]:
#Файл с каталогом
catalog_list = get_catalogs_wb()
save_excel(catalog_list,"1_catwb")
save_json(catalog_list,"1_catwb")
#Загрузка каталога для обработки
catwb = pd.read_excel('1_catwb.xlsx')

Все сохранено в 1_catwb.xlsx
Данные сохранены в 1_catwb.json


In [ ]:
#Загрузка товаров по категориям
error = []
maincat = []
n = 0
for i in range(0,len(catwb)):
    url = 'https://www.wildberries.ru'+catwb['category_url'][i]
    low_price = 1
    top_price = 1000000
    cat_id = catwb['categoty_id'][i]
    
    #Доп проверка на наличие категории в каталоге
    id_category, name_category, shard, query = search_category_in_catalog(url=url, catalog_list=catalog_list)
    print("Номер категории в списке:", i)
    try:
        data = get_content(shard=shard, query=query, low_price=low_price, top_price=top_price)
        save_json(data, f'Категория id {cat_id}')
        n+=1
    except:
        print(f'Данные по категории № {i} не собрались')
        try:
            print(f'Повторный сбор по категории № {i}')
            data = get_content(shard=shard, query=query, low_price=low_price, top_price=top_price)
            save_json(data, f'Категория id {cat_id}')
            n+=1
        except:
            print(f'ERROR, данные по категории № {i} не собраны')
            error.append(i)
            continue

найдено совпадение: Блузки и рубашки
Номер категории в списке: 0


  0%|          | 0/100 [00:00<?, ?it/s]

Данные сохранены в Категория id 8126.json
найдено совпадение: Брюки
Номер категории в списке: 1


  0%|          | 0/100 [00:00<?, ?it/s]

Данные сохранены в Категория id 8127.json
найдено совпадение: Верхняя одежда
Номер категории в списке: 2


...

  0%|          | 0/100 [00:00<?, ?it/s]

Данные сохранены в Категория id 60578.json
найдено совпадение: Мебель
Номер категории в списке: 367


  0%|          | 0/100 [00:00<?, ?it/s]

Данные по категории № 367 не собрались
Повторный сбор по категории № 367


  0%|          | 0/100 [00:00<?, ?it/s]

In [8]:
#Обработка ошибок        
for k in error:
    url = 'https://www.wildberries.ru'+catwb['category_url'][k]
    low_price = 1
    top_price = 1000000
    id_category, name_category, shard, query = search_category_in_catalog(url=url, catalog_list=catalog_list)
    print("Номер категории в списке:", k)
    cat_id = catwb['categoty_id'][k]
    try:
        data = get_content(shard=shard, query=query, low_price=low_price, top_price=top_price)
        save_json(data, f'Категория id {cat_id}')
        n+=1
    except:
        print(f'Данные по категории № {k} не собрались')
        try:
            print(f'Повторный сбор по категории № {k}')
            data = get_content(shard=shard, query=query, low_price=low_price, top_price=top_price)
            save_json(data, f'Категория id {cat_id}')
            n+=1
        except:
            print(f'{k} Основная категория, данные не собираются')
            maincat.append(k)
            continue
        
m = len(maincat)            
print(f'Собралось категорий {n}')
print(f'Количество категорий, которые не имеют данных {m}')

найдено совпадение: Белье
Номер категории в списке: 16


  0%|          | 0/100 [00:00<?, ?it/s]

Данные по категории № 16 не собрались
Повторный сбор по категории № 16


  0%|          | 0/100 [00:00<?, ?it/s]

16 Основная категория, данные не собираются
найдено совпадение: Большие размеры
Номер категории в списке: 29


...

1548 Основная категория, данные не собираются
найдено совпадение: Organic Altay
Номер категории в списке: 1549


  0%|          | 0/100 [00:00<?, ?it/s]

Данные по категории № 1549 не собрались
Повторный сбор по категории № 1549


  0%|          | 0/100 [00:00<?, ?it/s]

1549 Основная категория, данные не собираются
Собралось категорий 1498
Количество категорий, которые не имеют данных 52


In [9]:
#Создание нового спсика каталога с удаленнными основными категориями
catwb1 = catwb.drop(labels = maincat)
catwb1 = catwb1.drop(columns=catwb1.columns[0])
catwb2 = catwb1.reset_index(drop=True)

save_excel(catwb1,"1_catwb_new_w_old_index")
save_excel(catwb2,"1_catwb_new_w_new_index")

catwb = pd.read_excel('1_catwb_new_w_new_index.xlsx')

Все сохранено в 1_catwb_new_w_old_index.xlsx
Все сохранено в 1_catwb_new_w_new_index.xlsx


In [2]:
#Совмещение всех файлов
data_all = []
r = 0
for l in range (0,len(catwb)):
    cat_id = catwb['categoty_id'][l]
    try:
        with open( f'Категория id {cat_id}.json',encoding = 'UTF-8') as f:
            data = json.load(f)
            data_all.append(data)
        r += 1
    except:
        print(f'Категории {cat_id} нет в файлах')
        continue
            
save_json(data_all, '1_ВСЕ ДАННЫЕ')
print(f'Совмещенно файлов {r} из {n}')

Категории 63082 нет в файлах
Категории 10039 нет в файлах
Категории 63003 нет в файлах
Категории 8953 нет в файлах
Категории 8985 нет в файлах
Категории 16357 нет в файлах
Категории 9393 нет в файлах
Категории 128792 нет в файлах
Категории 128769 нет в файлах
Категории 128848 нет в файлах
Данные сохранены в 1_ВСЕ ДАННЫЕ.json
Совмещенно файлов 1488


In [ ]:
#Информация по продавцам/магазинам
shop_id=[]
z=0
p=0

for i in range (0,len(catwb)):
    cat_id = catwb['categoty_id'][i]
    url = 'https://www.wildberries.ru'+ catwb['category_url'][i]    
    try:
        with open(f'Категория id {cat_id}.json',encoding = 'UTF-8') as f:
            data = json.load(f)
            for d in data:
                shop = d['id магазина']
                shop_id.append(shop)
            z +=1
    except:
        print(f'Не нашел id {cat_id}')
        p+=1
        continue
        
shop_data = list(set(shop_id))
save_json(shop_data,'1_id shop')

sup_inf = supplier_inf(shop_data)
save_json(sup_inf,'1_supplier inf')